# Practicumopdrachten Week 2.2 (eerste kans)

## Dependencies importeren

Importeer in het codeblok hieronder de packages die worden gebruikt om data in te lezen. Geef er ook de gebruikelijke aliassen aan.<br>
N.B.: de 2 reeds geschreven coderegels zorgen ervoor dat eventuele warnings, die code-outputs lelijker maken, uitgezet worden.

In [ ]:
import pandas as pd
import sqlite3 as s3
import warnings
warnings.simplefilter('ignore')

Zet het bestand go_sales.sqlite in hetzelfde mapje als dit notebook.

## Databasetabel inlezen

Creëer een databaseconnectie met het bestand go_sales.sqlite.

In [ ]:
go_sales = s3.connect('go_sales.sqlite')

Gebruik de onderstaande sql_query om te achterhalen welke databasetabellen in go_sales zitten.

In [ ]:
sql_query = "SELECT name FROM sqlite_master WHERE type='table';"
#Vul dit codeblok verder in

pd.read_sql_query(sql_query, go_sales)

Gebruik de gecreëerde databaseconnectie om de resultaten van de volgende query in een DataFrame te zetten:<br>
*SELECT * FROM sales_staff* 

In [ ]:
staff = pd.read_sql_query('SELECT * FROM sales_staff', go_sales)

## DataFrames uitsplitsen en opbouwen met Series

De volgende 5 kolommen hebben betrekking op de contactdetails van elke medewerker in dit DataFrame:
- SALES_STAFF_CODE
- WORK_PHONE
- EXTENSION
- FAX
- EMAIL

Maak van elk van deze 5 kolommen een serie.

In [ ]:
print(staff['SALES_STAFF_CODE'])
print(staff['WORK_PHONE'])
print(staff['EXTENSION'])
print(staff['FAX'])
print(staff['EMAIL'])

Zet allevijf gecreëerde series als kolommen naast elkaar in een DataFrame (*contact_details*). Gebruik pd.concat(...)<br>
Hulpvraag: welke waarde geef je aan de axis-parameter?

In [ ]:
contact_details = pd.concat([staff['SALES_STAFF_CODE'], staff['WORK_PHONE'], staff['EXTENSION'], staff['FAX'], staff['EMAIL']], axis=1)

## Series en DataFrames maken vanuit lists en dictionaries

Met .head(*getal*) kan je de bovenste *getal* rijen van een tabel selecteren.<br>
Selecteer op deze manier de bovenste 5 rijen van *contact_details*.<br>
Sla dit resultaat op in een nieuw DataFrame.

In [ ]:
top5 = contact_details.head(5)

Aan deze 10 rijen met contactdetails willen we 3 kolommen toevoegen: 'FIRST_LANGUAGE', 'SECOND_LANGUAGE' & 'THIRD_LANGUAGE'.<br>
Iedereens 'First Language' is Engels, afgekort 'EN'. Maak een lijst waarin 5x 'EN' staat.<br>
Converteer deze lijst vervolgens naar een serie en voeg deze horizontaal samen met het resultaat van de vorige opdracht.<br>
Vergeet niet een passende naam te geven aan de nieuw ontstane kolom.

In [ ]:
fl_lijst = ['EN', 'EN', 'EN', 'EN', 'EN']
fl_serie = pd.Series(fl_lijst)
top5['FIRST_LANGUAGE'] = fl_serie

Maak nu de tweede kolom ('SECOND_LANGUAGE'). Maak daarvoor een dictionary, met daarin...
- Als keys: alle indexen uit het resultaat van het vorige codeblok.
- Als values: bij de eerste 3 elementen 'FR' (Frankrijk), bij de laatste 2 'DE' (Duitsland).

Maak vervolgens ook hier weer een serie van en voeg ook deze weer horizontaal samen met het rsultaat van de vorige opdracht.<br>
Vergeet niet een passende naam te geven aan de nieuw ontstane kolom.

In [ ]:
sl_dict = { 0 : 'FR', 1 : 'FR', 2 : 'FR', 3 : 'DE', 4 : 'DE' }
sl_serie = pd.Series(sl_dict)
top5['SECOND_LANGUAGE'] = sl_serie

Maak ten slotte de derde kolom ('THIRD_LANGUAGE') door een dictionary te maken met daarin...
- Als key: de naam van de nieuwe kolom. Zie je het verschil qua keys met de vorige opdracht?
- Als waarde: een lijst met daarin 'NL', 'NL', 'JPN', 'JPN', 'KOR'.

Converteer deze dictionary nu naar een DataFrame en voeg deze horizontaal samen met het resultaat van de vorige opdracht.<br>
Waarom hoef je hierna de nieuw ontstane kolom geen passende naam meer te geven?

In [ ]:
tl_dict = { 'THIRD_LANGUAGE' : ['NL', 'NL', 'JPN', 'JPN', 'KOR'] }
tl_dataframe = pd.DataFrame(tl_dict)
top5['THIRD_LANGUAGE'] = tl_dataframe
pd.concat([top5['SALES_STAFF_CODE'], top5['WORK_PHONE'], top5['EXTENSION'], top5['FAX'], top5['EMAIL'], top5['FIRST_LANGUAGE'], top5['SECOND_LANGUAGE'], top5['THIRD_LANGUAGE']], axis = 1)

## Data toevoegen

### Rijen

Gebruik de originele databasetabel *sales_staff*.<br>
Voeg een extra rij toe met eigen invulling. Zorg ervoor dat de index netjes doorloopt.<br>
Hulpvraag: welke waarde geef je nu aan axis?

In [ ]:
new_staff_dict = {
    'SALES_STAFF_CODE' : 12345,
    'FIRST_NAME' : 'Kippie',
    'LAST_NAME' : 'Aapie',
    'POSITION_EN' : 'Level 10 Raid Boss',
    'WORK_PHONE' : '112',
    'EXTENSION' : '999',
    'FAX' : '123456789',
    'EMAIL' : 'test@example.com',
    'DATE_HIRED' : '14-2-2024',
    'SALES_BRANCH_CODE' : 33,
    'TRIAL888' : 'T'
}

staff = pd.concat([staff, pd.DataFrame(new_staff_dict, index=[0])], ignore_index=True)

### Kolommen

Voeg een kolom *FULL_NAME* toe die de waarden van *FIRST_NAME* en *LAST_NAME* achter elkaar zet, gescheiden door een spatie.

In [ ]:
staff['FULL_NAME'] = staff['FIRST_NAME'] + " " + staff['LAST_NAME']

## Data wijzigen

### Datatypen

Door het attribuut .dtypes van een DataFrame op te vragen krijg je een serie die per kolom het datatype weergeeft. Doe dit bij de originele databasetabel *sales_staff*

In [ ]:
staff = pd.read_sql_query('SELECT * FROM sales_staff', go_sales)
staff.dtypes

Hier valt op dat elke kolom het datatype 'object' heeft: Python leest dus alles als string. Wiskundige operaties zijn hierdoor niet mogelijk.<br>
We kunnen proberen om kolommen met getallen, bijvoorbeeld de 'extension', te converteren naar een int. Zie onderstaande code.

In [ ]:
staff['EXTENSION'] = staff['EXTENSION'].astype('Int64')
staff.dtypes

Dit lukt echter niet, omdat er in de kolom 'EXTENSION' lege waarden zitten die niet geconverteerd kunnen worden naar een int.<br>
Wél kan je deze naar een float converteren, zie onderstaande code:

In [ ]:
staff['EXTENSION'] = staff['EXTENSION'].astype(float)
staff.dtypes

Als we in de rij van 'EXTENSION' kijken zien we dat de conversie van het datatype nu is gelukt.<br>
Dit is <b>randvoorwaardelijk</b> voor het uitvoeren van wiskundige operaties.<br>

### Rijen

Zorg er nu voor dat bij alle extensions 1 wordt opgeteld.

In [ ]:
staff['EXTENSION'] += 1

Elke 'Branch Manager' wordt nu 'General Manager'. Schrijf code zodat deze wijziging doorgevoerd wordt in het DataFrame.

In [ ]:
staff = staff.replace('Branch Manager', 'General Manager')

### Kolommen

Verander de kolomnaam 'POSITION_EN' naar 'POSITION'.

In [ ]:
staff = staff.rename(columns = {'POSITION_EN' : 'POSITION'})

## Data verwijderen

### Rijen

De medewerkers op indexen 99, 100 en 101 hebben helaas ontslag genomen.<br>
Verwijder de bijbehorende rijen uit het DataFrame. Gebruik slechts één keer de .drop()-methode.

In [ ]:
staff = staff.drop([99, 100, 101], axis=0)

### Kolommen

Faxen zijn inmiddels ouderwets: niemand gebruikt zijn/haar faxnummer nog.<br>
Verwijder de bijbehorende kolom uit het DataFrame.

In [ ]:
staff = staff.drop('FAX', axis = 1)